In [2]:
import pandas as pd
import seaborn as sb

#df = pd.read_csv('forestfires.csv')
df_portugal = pd.read_csv('data/Portugal_Dataset.csv')
df_portugal
#sb.heatmap(df.corr(), annot = True, fmt='.1g')


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.00
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.00
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.00
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.00
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.00


In [3]:
#Creates a dataframe containing california counties and their relative biomass(dead & alive) in tons
#4 counties unaccounted for in dead biomass so it's assumed that these areas have negligible dead biomass
df61 = pd.read_csv('data/data61.txt', delim_whitespace=True)
df61 = df61[['County', 'All_Total']]
df61 = df61.rename(columns={'All_Total': "Alive_Biomass_Tons"})
df62 = pd.read_csv('data/data62.txt', delim_whitespace=True)
df62 = df62[['County', 'All_Total']]
df62 = df62.rename(columns={'All_Total': "Dead_Biomass_Tons"})
for county in df61['County']:
    if county not in df62['County'].values:
        df62 = df62.append({'County': county,"Dead_Biomass_Tons": 0}, ignore_index=True)
df62 = df62.sort_values(by ='County', ascending = 1)
dfMass = pd.merge(df61,df62, on = ['County'])

for county in dfMass['County']:
    if "_" in county:
        fix = county.replace("_", " ")
        dfMass = dfMass.replace(county, fix)
dfMass

,County,Alive_Biomass_Tons,Dead_Biomass_Tons
0,Alameda,"5,363.1",205.4
1,Alpine,"21,755.0","3,009.8"
2,Amador,"13,987.8","1,506.8"
3,Butte,"46,750.7","3,181.1"
4,Calaveras,"27,078.1","1,732.0"
5,Colusa,"5,376.2",363.1
6,Contra Costa,"2,799.0",10.4
7,Del Norte,"81,591.3","8,065.7"
8,El Dorado,"73,636.9","7,813.7"
9,Fresno,"80,129.4","8,425.4"


In [4]:
# Remove irrelevant columns
# Location may be irrelevant because all are xy coordinates for Montesinho park
columns_to_keep_portugal = [
    'month', 
    'day', 
    'temp', 
    'RH',  # relative humidity
    'wind',  # wind speed
    'rain',  # rain amount
    'area'  # burned area
]

df_portugal = df_portugal[columns_to_keep_portugal]
df_portugal.shape


(517, 7)

In [5]:
# Removing all missing data
df_portugal = df_portugal.dropna()
print(df_portugal.shape)

(517, 7)


In [6]:
# Show updated dataset
df_portugal
df_portugal.to_csv('data/Portugal_Dataset_clean.csv')

In [7]:
import pandas as pd
import seaborn as sb

df_cal_weather = pd.read_csv('data/California_Weather.csv' )

#df_cal_weather

In [8]:
# Remove irrelevant columns
columns_to_keep_cal_weather = [
    'Stn Id',
    'Stn Name',
    'CIMIS Region', 
    'Date', 
    'Precip (in)',
    'Avg Air Temp (F)',
    'Avg Rel Hum (%)',
    'Avg Wind Speed (mph)',
    'Wind Run (miles)'
]

df_cal_weather = df_cal_weather[columns_to_keep_cal_weather]
df_cal_weather.shape

(128125, 9)

In [9]:
# Removing all missing data
df_cal_weather = df_cal_weather.dropna()
print(df_cal_weather.shape)

(128112, 9)


In [10]:
# Show updated dataset
df_cal_weather
df_cal_weather.to_csv('data/California_Weather_clean.csv')

In [11]:
import pandas as pd
import seaborn as sb

df_cal_fire = pd.read_csv('data/California_Fire_Incidents.csv')
df_cal_fire


,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0
5,22992.0,False,Sierra National Forest,NaN,2013,False,/incidents/2013/7/22/aspen-fire/,NaN,NaN,Fresno,...,"217 Aspen Fire, July 2013, Big Creek, Fresno ...",2013-07-22T22:15:00Z,Finalized,NaN,NaN,NaN,NaN,bee8c339-4f26-4b78-a5b4-a8a0ebdb8786,2013-09-24T20:15:00Z,NaN
6,20292.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/8/7/silver-fire/,Firefighters closed the containment lines toda...,Hwy 243 remains closed between Twin Pines Road...,Riverside,...,"Silver Fire, August 2013, Popet Flats Road, Hi...",2013-08-07T14:05:00Z,Finalized,8.0,40.0,NaN,NaN,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,2013-08-12T18:00:00Z,20.0
7,14754.0,False,Klamath National Forest,NaN,2013,False,/incidents/2013/7/31/salmon-river-complex/,NaN,NaN,Siskiyou,...,"210 Salmon River Complex, August 2013, July 20...",2013-07-31T22:00:00Z,Finalized,NaN,NaN,NaN,NaN,ba76c009-09c9-497c-b923-1ac8cf05c20d,2013-08-31T06:45:00Z,NaN
8,12503.0,False,Six Rivers National Forest,NaN,2013,False,/incidents/2013/8/10/corral-complex/,NaN,NaN,Humboldt,...,"Corral Complex, August 2013, Tish Tang Ridge, ...",2013-08-10T11:40:00Z,Finalized,NaN,NaN,NaN,NaN,f3dcbca8-f8ed-46d0-8f6e-9c7fc2fb03e6,2013-08-12T12:00:00Z,NaN
9,11429.0,False,CAL FIRE Tehama-Glenn Unit,NaN,2013,True,/incidents/2013/8/23/deer-fire/,Fire suppression repair will continue througho...,NaN,Tehama,...,"Deer Fire, August 2013, Tehama County, Deer Cr...",2013-08-23T14:15:00Z,Finalized,NaN,NaN,NaN,NaN,956dbcf6-db40-4b61-b235-4ede14738f1e,2013-08-29T16:45:00Z,8.0


In [12]:
# Remove irrelevant columns
columns_to_keep_cal_fire = [
    'AcresBurned',
    'ArchiveYear',
    'CalFireIncident',
    'Counties', 
    'Extinguished',
    'Location',  
    'Started'
]

df_cal_fire = df_cal_fire[columns_to_keep_cal_fire]
df_cal_fire.shape

(1636, 7)

In [13]:
# Removing all missing data
df_cal_fire = df_cal_fire.dropna()
print(df_cal_fire.shape)

(1577, 7)


In [14]:
# Show updated dataset
df_cal_fire
df_cal_fire.to_csv('data/California_Fire_Incidents_clean.csv')

In [15]:
# Combine the data from California Weather and California Fire
# Combine based on location and Date

# Split the Date column from California Weather into month / year
df_cal_weather['month'] = df_cal_weather['Date'].str.split('/').str[0].astype('int')
df_cal_weather['year'] = df_cal_weather['Date'].str.split('/').str[2].astype('int')
df_cal_weather['day'] = df_cal_weather['Date'].str.split('/').str[1].astype('int')



df_cal_weather

,Stn Id,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles),month,year,day
0,2,FivePoints,San Joaquin Valley,1/1/2018,0.00,47.8,65.0,3.3,78.3,1,2018,1
1,2,FivePoints,San Joaquin Valley,1/2/2018,0.00,47.2,67.0,3.1,74.5,1,2018,2
2,2,FivePoints,San Joaquin Valley,1/3/2018,0.00,49.9,68.0,4.5,107.5,1,2018,3
3,2,FivePoints,San Joaquin Valley,1/4/2018,0.01,56.8,74.0,5.8,140.2,1,2018,4
4,2,FivePoints,San Joaquin Valley,1/5/2018,0.00,59.8,72.0,4.2,101.4,1,2018,5
5,2,FivePoints,San Joaquin Valley,1/6/2018,0.04,55.2,90.0,6.7,162.0,1,2018,6
6,2,FivePoints,San Joaquin Valley,1/7/2018,0.00,56.4,74.0,3.5,83.3,1,2018,7
7,2,FivePoints,San Joaquin Valley,1/8/2018,0.38,55.6,92.0,5.1,122.0,1,2018,8
8,2,FivePoints,San Joaquin Valley,1/10/2018,0.00,54.1,75.0,5.7,137.6,1,2018,10
9,2,FivePoints,San Joaquin Valley,1/11/2018,0.00,52.9,80.0,2.6,61.8,1,2018,11


In [16]:
# Split the Started column from California Weather into month / year
df_cal_fire['year'] = df_cal_fire['Started'].str.split('-').str[0].astype('int')
df_cal_fire['month'] = df_cal_fire['Started'].str.split('-').str[1].astype('int')

df_cal_fire


,AcresBurned,ArchiveYear,CalFireIncident,Counties,Extinguished,Location,Started,year,month
0,257314.0,2013,True,Tuolumne,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,2013-08-17T15:25:00Z,2013,8
1,30274.0,2013,True,Los Angeles,2013-06-08T18:30:00Z,Angeles National Forest,2013-05-30T15:28:00Z,2013,5
2,27531.0,2013,True,Riverside,2013-07-30T18:00:00Z,Hwy 243 & Hwy 74 near Mountain Center,2013-07-15T13:43:00Z,2013,7
3,27440.0,2013,False,Placer,2013-08-30T08:00:00Z,"Deadwood Ridge, northeast of Foresthill",2013-08-10T16:30:00Z,2013,8
4,24251.0,2013,True,Ventura,2013-05-11T06:30:00Z,Southbound Highway 101 at Camarillo Springs Ro...,2013-05-02T07:01:00Z,2013,5
5,22992.0,2013,False,Fresno,2013-09-24T20:15:00Z,Seven miles north of Big Creek,2013-07-22T22:15:00Z,2013,7
6,20292.0,2013,True,Riverside,2013-08-12T18:00:00Z,"Poppet Flats Rd near Hwy 243, south of Banning",2013-08-07T14:05:00Z,2013,8
7,14754.0,2013,False,Siskiyou,2013-08-31T06:45:00Z,"North Fork of the Salmon River, West of Sawyer...",2013-07-31T22:00:00Z,2013,7
8,12503.0,2013,False,Humboldt,2013-08-12T12:00:00Z,Tish Tang Ridge east of Hoopa Valley Reservation,2013-08-10T11:40:00Z,2013,8
9,11429.0,2013,True,Tehama,2013-08-29T16:45:00Z,"Near Deer Creek, 12 miles east of Los Molinos.",2013-08-23T14:15:00Z,2013,8


In [17]:
# Get the unique Regions/Counties of weather/fire and use 'str.replace()' to merge identical values

# Cleaning up Cal_Fire Counties to match Cal_weather Regions Choices

# San Francisco Bay
sf_bay = ['Alameda', 'Amador', 'Marin', 'Napa', 'San Mateo', 'Santa Clara', 'Sonoma']
# ?
cent_cal = ['Fresno', 'Monterey','San Joaquin Valley']
# Sacramento Valley
sac_valley = ['Butte', 'Sacramento Valley', 'Sutter', 'Yuba']
# North Coast Valleys
north_coast = ['Humboldt','Mendocino']
# ?
in_so_cal = ['Kern', 'San Bernardino']
# Los Angeles Basin
la_basin = ['Los Angeles']
# ?
great_basin = ['Modoc']
# Sierra Foothill
sierra_nv = ['Nevada', 'Sierra', 'State of Nevada']
# South Coast Valleys
so_coast = ['Orange', 'San Diego']
# Central Coast Valleys
cent_coast = ['San Luis Obispo', 'Santa Barbara', 'Santa Cruz']
# ?
shasta_cas = [ 'Shasta', 'Trinity']

# Unknown:
# Alpine, Calaveras, Colusa, Contra Costa, Del Norte, El Dorado, Glenn, Inyo, Kings, Lake, Lassen, Madera
# Mariposa, Merced, Mexico, Mono, Placer, Plumas, Riverside, San Benito, Siskiyou, Solano, Stanislaus
# State of Oregon, Tehama, Tulare, Tuolumne, Ventura, Yolo,


df_cal_fire['Counties'] = df_cal_fire['Counties'].str.replace('Alameda', 'San Francisco Bay')

regions_list = {'SanFranciscoBay': sf_bay, 'CentralCal': cent_cal, 'SacramentoValley': sac_valley, 
                'NorthCoastValleys': north_coast, 'InlandSoCal': in_so_cal, 'LosAngelesBasin': la_basin, 
                'GreatBasin': great_basin, 'SierraFoothill': sierra_nv, 'SouthCoastValleys': so_coast, 
                'CentralCoastValleys': cent_coast, 'ShastaCascade': shasta_cas}

for region_name in regions_list:
    for county in regions_list[region_name]:
        # FIXME
        df_cal_fire['Counties'] = df_cal_fire['Counties'].str.replace(county, region_name)

unique_regions = (df_cal_weather['CIMIS Region'].unique())
print(sorted(unique_regions))
unique_counties = (df_cal_fire['Counties'].unique())
print(sorted(unique_counties))



['Bishop', 'Central Coast Valleys', 'Eastern Sierra Basin & Range', 'Imperial/Coachella Valley', 'Los Angeles Basin', 'Monterey Bay', 'North Coast Valleys', 'Northeast Plateau', 'Sacramento Valley', 'San Bernardino', 'San Francisco Bay', 'San Joaquin Valley', 'Sierra Foothill', 'South Coast Valleys']
['Alpine', 'Calaveras', 'CentralCal', 'CentralCoastValleys', 'Colusa', 'Contra Costa', 'Del Norte', 'El Dorado', 'Glenn', 'GreatBasin', 'InlandSoCal', 'Inyo', 'Kings', 'Lake', 'Lassen', 'LosAngelesBasin', 'Madera', 'Mariposa', 'Merced', 'Mexico', 'Mono', 'NorthCoastValleys', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'SacramentoValley', 'San Benito', 'San Francisco Bay', 'San Joaquin', 'SanFranciscoBay', 'ShastaCascade', 'SierraFoothill', 'SierraFoothillFoothill', 'Siskiyou', 'Solano', 'SouthCoastValleys', 'Stanislaus', 'State of Oregon', 'State of SierraFoothillFoothill', 'Tehama', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo']


In [18]:
#import CIMIS station data, drop Stn Name(duplicate with CA weather) and Status(irrelevant)
df_cimis = pd.read_csv('data/cimis_stations.csv', names=['Stn Id', 'Stn Name', 'Counties', 'Status'])
df_cimis = df_cimis[['Stn Id','Counties']]

#combine CIMIS data with CA weather data (CIMIS data has counties that are in the same format as in CA fire)
df_weather_with_counties = pd.merge(df_cal_weather,df_cimis, on = ['Stn Id'])

#combine matching entries of CA fire and CA weather by County, month, and year
df_CA_merged = pd.merge(df_cal_fire,df_weather_with_counties, how = 'inner', on = ['Counties','month','year'])

df_CA_merged

,AcresBurned,ArchiveYear,CalFireIncident,Counties,Extinguished,Location,Started,year,month,Stn Id,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles),day
0,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/1/2018,0.00,82.7,52.0,4.8,114.4,1
1,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/2/2018,0.00,75.8,67.0,5.0,119.4,2
2,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/3/2018,0.00,73.9,63.0,5.0,119.9,3
3,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/4/2018,0.00,67.6,72.0,5.6,135.6,4
4,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/5/2018,0.00,71.0,65.0,3.8,90.6,5
5,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/6/2018,0.00,74.0,67.0,2.9,70.7,6
6,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/7/2018,0.00,76.2,63.0,3.4,81.7,7
7,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/8/2018,0.00,74.7,66.0,4.1,99.3,8
8,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/9/2018,0.00,75.4,59.0,3.9,92.8,9
9,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,250,Williams,Sacramento Valley,7/10/2018,0.00,79.6,50.0,3.7,88.4,10


Need to find more Weather to make more data. Also, if we can able to find other feature like the "Size of the forest", It Will Be Much Better

Here is the way to change the merged dataframe to the DATAFRAME WE ACTAULLY WANT

this function returns a dictionary containing the average weather attributes in over a fires burn time
"start" and "end" should take the form passed from the columns: Started and Extingushed (ex. parameter: 2019-01-04T09:33:00Z)
"stn" is the station ID

In [19]:

def getWeather(start, end, stn):
    last_days = {1:31, 2:28, 3:31, 4:30,5:31 ,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    year = int(start[0:4])
    month = int(start[5:7])
    day = int(start[8:10])

    if(day == 29 and month == 2):#incase of leap year
        day = 1
        month = 3
    Emonth = int(end[5:7])
    Eday = int(end[8:10])
    if(Eday == 29 and Emonth == 2):#increase of leap year
        Eday = 1
        Emonth = 3
    days = 0
    aRain = 0
    aTemp = 0
    aHum = 0 
    aWind = 0
    exist_day = 0
    while True:
        dayWeather = df_cal_weather.loc[(df_cal_weather['Stn Id'] == stn) & (df_cal_weather['month'] == month) & (df_cal_weather['day'] == day) & (df_cal_weather['year'] == year)]
        if(dayWeather.empty):
            print("No weather data for %d/%d/%d at station: %d" %(month, day, year, stn))
        else:
            try: 
                aRain += float(dayWeather['Precip (in)'].values)
                aTemp += float(dayWeather['Avg Air Temp (F)'].values)
                aHum  += float(dayWeather['Avg Rel Hum (%)'].values)
                aWind += float(dayWeather['Avg Wind Speed (mph)'].values)

                exist_day += 1
            except:
                pass
        
        days +=1


        if (day == Eday) and (month == Emonth):
            break
            
        day += 1
        
        if (days > 500):
            return
        
        
        if(day > last_days.get(month)):
            day = 1
            month += 1
            if(month == 13):
                month = 1
                year += 1 
        
    
    if exist_day != 0:
        aRain /= exist_day
        aTemp /= exist_day
        aHum /= exist_day
        aWind /= exist_day
    else:
        exist_day+=1
        aRain /= exist_day
        aTemp /= exist_day
        aHum /= exist_day
        aWind /= exist_day
        
    avg_weather= {"Days Burned": days,"Precipitation" : round(aRain, 6), "Temperature" : round(aTemp,6), "Humidity" : round(aHum,6), "Wind Speed" : round(aWind,6)}
    
    return avg_weather

Important_features IS THE FEATURE IN df_cal_fire, Weather_feature is the FEATURE IN df_cal_weather

Because we merged them together we only need to extract them from df_CA_merged based on getWeather

In [20]:
Important_features = [
    'AcresBurned',
    'ArchiveYear',
    'CalFireIncident',
    'Counties', 
    'Extinguished',
    'Started',
    "Stn Id",
    "Precip (in)",
    "CIMIS Region"
]
Weather_feature = [
   'Days Burned', 'Precipitation', 'Temperature', 'Humidity', 'Wind Speed',
]

In [21]:
Unique_fire_location = df_CA_merged.Location.unique()

CA_fire_actual_DF = {}
for i in Important_features:
    CA_fire_actual_DF[i] = []

for i in Weather_feature:
    CA_fire_actual_DF[i] = []
    
for i in range(len(Unique_fire_location)):
    current_unique_event = df_CA_merged.loc[df_CA_merged['Location'] == Unique_fire_location[i]].iloc[0]
    for j in Important_features:
        CA_fire_actual_DF[j].append(current_unique_event[j])

  
    AvgWeather = getWeather(current_unique_event["Started"],current_unique_event["Extinguished"],current_unique_event["Stn Id"])

    
    for j in Weather_feature:
        CA_fire_actual_DF[j].append(AvgWeather[j])
        
        
for i in CA_fire_actual_DF:
    print(len(CA_fire_actual_DF[i]))

No weather data for 7/19/2018 at station: 250
No weather data for 7/26/2018 at station: 250
No weather data for 11/3/2018 at station: 6
No weather data for 11/3/2018 at station: 6
No weather data for 7/27/2018 at station: 57
No weather data for 8/15/2018 at station: 57
No weather data for 8/16/2018 at station: 57
No weather data for 8/17/2018 at station: 57
No weather data for 8/18/2018 at station: 57
No weather data for 7/24/2018 at station: 57
No weather data for 7/25/2018 at station: 57
No weather data for 7/26/2018 at station: 57
No weather data for 7/27/2018 at station: 57
No weather data for 8/15/2018 at station: 57
No weather data for 8/16/2018 at station: 57
No weather data for 8/17/2018 at station: 57
No weather data for 8/18/2018 at station: 57
No weather data for 7/22/2018 at station: 57
No weather data for 7/23/2018 at station: 57
No weather data for 7/24/2018 at station: 57
No weather data for 7/25/2018 at station: 57
No weather data for 7/26/2018 at station: 57
No weather

No weather data for 7/18/2018 at station: 86
No weather data for 12/1/2018 at station: 86
No weather data for 12/23/2018 at station: 86
No weather data for 1/29/2018 at station: 152
No weather data for 3/31/2018 at station: 152
No weather data for 4/9/2018 at station: 152
No weather data for 5/29/2018 at station: 152
No weather data for 7/6/2018 at station: 152
No weather data for 7/7/2018 at station: 152
No weather data for 10/30/2019 at station: 152
No weather data for 10/31/2019 at station: 152
No weather data for 11/1/2019 at station: 152
No weather data for 11/2/2019 at station: 152
No weather data for 10/31/2019 at station: 152
No weather data for 11/1/2019 at station: 152
No weather data for 11/2/2019 at station: 152
No weather data for 11/3/2019 at station: 152
No weather data for 11/4/2019 at station: 152
No weather data for 9/6/2019 at station: 13
No weather data for 6/8/2019 at station: 6
No weather data for 6/9/2019 at station: 6
No weather data for 6/10/2019 at station: 6


Here is the actually DATAFRAME WE WANT!

In [33]:
CA_fire_actual_DF = pd.DataFrame(CA_fire_actual_DF)
CA_fire_actual_DF.head(3)

,AcresBurned,ArchiveYear,CalFireIncident,Counties,Stn Id,Precip (in),CIMIS Region,Days Burned,Precipitation,Temperature,Humidity,Wind Speed
0,410203.0,2018,True,Colusa,250,0.0,Sacramento Valley,162,0.019444,60.873457,64.654321,4.472840
1,48920.0,2018,True,Colusa,250,0.0,Sacramento Valley,162,0.019444,60.873457,64.654321,4.472840
2,75.0,2018,True,Colusa,250,0.0,Sacramento Valley,185,0.017213,62.727869,64.683060,4.425137


Then we have to do the onehot encoder and standardize

In [23]:
#drop the uselss feature first
CA_fire_actual_DF = CA_fire_actual_DF.drop(["Extinguished","Started"], axis=1)

In [34]:
from sklearn.preprocessing import  OneHotEncoder
def Encode_Dataframe(df, labels):
    enc = OneHotEncoder(sparse = False) 
    new_df = {}
    for i in labels:
        new_df[i] = df[i]
    new_df = pd.DataFrame(new_df)
    ans = enc.fit_transform(new_df)
    
    for i in labels:
        df = df.drop(i, axis = 1)
    new_df = pd.DataFrame(ans)
    for i in new_df:
        df[i] = new_df[i]
    return df


label_feature = ["CalFireIncident", "Counties", "Stn Id", "CIMIS Region"]
df = CA_fire_actual_DF.copy()
df = Encode_Dataframe(df, label_feature)
df.head(3)

,AcresBurned,ArchiveYear,Precip (in),Days Burned,Precipitation,Temperature,Humidity,Wind Speed,0,1,...,40,41,42,43,44,45,46,47,48,49
0,410203.0,2018,0.0,162,0.019444,60.873457,64.654321,4.472840,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,48920.0,2018,0.0,162,0.019444,60.873457,64.654321,4.472840,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,75.0,2018,0.0,185,0.017213,62.727869,64.683060,4.425137,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


sklearn maxminScaler to standard the data    

In [35]:
from sklearn import preprocessing
names = [  'ArchiveYear',   'Precip (in)',   'Days Burned',
            'Precipitation',   'Temperature',      'Humidity',    'Wind Speed',]
Nomalized_df = {}
for i in names:
    Nomalized_df[i] = df[i]

scaler = preprocessing.MinMaxScaler()                  
Nomalized_df = scaler.fit_transform(pd.DataFrame(Nomalized_df))
Nomalized_df = pd.DataFrame(Nomalized_df, columns=names)
Nomalized_df.head(3)

,ArchiveYear,Precip (in),Days Burned,Precipitation,Temperature,Humidity,Wind Speed
0,0.0,0.0,0.448468,0.075170,0.694110,0.769694,0.286721
1,0.0,0.0,0.448468,0.075170,0.694110,0.769694,0.286721
2,0.0,0.0,0.512535,0.066545,0.715255,0.770036,0.283663


insert back to the df

In [36]:
for i in names:
    df[i] = Nomalized_df[i]
df.head(3)

,AcresBurned,ArchiveYear,Precip (in),Days Burned,Precipitation,Temperature,Humidity,Wind Speed,0,1,...,40,41,42,43,44,45,46,47,48,49
0,410203.0,0.0,0.0,0.448468,0.075170,0.694110,0.769694,0.286721,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,48920.0,0.0,0.0,0.448468,0.075170,0.694110,0.769694,0.286721,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,75.0,0.0,0.0,0.512535,0.066545,0.715255,0.770036,0.283663,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
df.to_csv('data/Ca_fire_weather.csv')